In [1]:
from functools import reduce

import pandas as pd
import numpy as np

import pickle

In [2]:
preds_path_ls = [
    ('raw 0 shot',   r'C:\Users\Vladimir\PycharmProjects\OrthogonalFineTune\test_preds\Raw_0sh.bin'),
    ('raw 5 shot',   r'C:\Users\Vladimir\PycharmProjects\OrthogonalFineTune\test_preds\Raw_5sh.bin'),
    ('LoRA 0 shot',  r'C:\Users\Vladimir\PycharmProjects\OrthogonalFineTune\test_preds\LoRA-cfg17_0sh.bin'),
    ('LoRA 5 shot',  r'C:\Users\Vladimir\PycharmProjects\OrthogonalFineTune\test_preds\LoRA-cfg17_5sh.bin'),
    ('GSOFT_2 0 shot', r'C:\Users\Vladimir\PycharmProjects\OrthogonalFineTune\test_preds\GSOFT-cfg2_0sh.bin'),
    ('GSOFT_2 5 shot', r'C:\Users\Vladimir\PycharmProjects\OrthogonalFineTune\test_preds\GSOFT-cfg2_5sh.bin'),
    ('GSOFT_3 0 shot', r'C:\Users\Vladimir\PycharmProjects\OrthogonalFineTune\test_preds\GSOFT-cfg3_0sh.bin'),
    ('GSOFT_3 5 shot', r'C:\Users\Vladimir\PycharmProjects\OrthogonalFineTune\test_preds\GSOFT-cfg3_5sh.bin'),
    ('GSOFT_4 0 shot', r'C:\Users\Vladimir\PycharmProjects\OrthogonalFineTune\test_preds\GSOFT-cfg4_0sh.bin'),
    ('GSOFT_4 5 shot', r'C:\Users\Vladimir\PycharmProjects\OrthogonalFineTune\test_preds\GSOFT-cfg4_5sh.bin'),
    ('GSOFT_5 0 shot', r'C:\Users\Vladimir\PycharmProjects\OrthogonalFineTune\test_preds\GSOFT-cfg5_0sh.bin'),
    ('GSOFT_5 5 shot', r'C:\Users\Vladimir\PycharmProjects\OrthogonalFineTune\test_preds\GSOFT-cfg5_5sh.bin'),
]

In [3]:
preds_dfs = {}

for run_name, preds_path in preds_path_ls:
    with open(preds_path, 'rb') as f:
        preds_dfs[run_name] = pickle.load(file=f)

        print(f"{run_name:16}: {preds_dfs[run_name].shape=}")

raw 0 shot      : preds_dfs[run_name].shape=(14042, 6)
raw 5 shot      : preds_dfs[run_name].shape=(14042, 6)
LoRA 0 shot     : preds_dfs[run_name].shape=(14042, 6)
LoRA 5 shot     : preds_dfs[run_name].shape=(14042, 6)
GSOFT_2 0 shot  : preds_dfs[run_name].shape=(14042, 6)
GSOFT_2 5 shot  : preds_dfs[run_name].shape=(14042, 6)
GSOFT_3 0 shot  : preds_dfs[run_name].shape=(14042, 6)
GSOFT_3 5 shot  : preds_dfs[run_name].shape=(14042, 6)
GSOFT_4 0 shot  : preds_dfs[run_name].shape=(14042, 6)
GSOFT_4 5 shot  : preds_dfs[run_name].shape=(14042, 6)
GSOFT_5 0 shot  : preds_dfs[run_name].shape=(14042, 6)
GSOFT_5 5 shot  : preds_dfs[run_name].shape=(14042, 6)


In [4]:
preds_dfs['raw 0 shot'].head()

,generated_text,subject,pred,true,corr,category
0,B,abstract_algebra,B,B,1,STEM
1,A,abstract_algebra,A,C,0,STEM
2,D,abstract_algebra,D,D,1,STEM
3,C,abstract_algebra,C,B,0,STEM
4,B,abstract_algebra,B,B,1,STEM


In [5]:
acc_by_subjects = pd.DataFrame({
    'subject': list(set.union(
        *map(
            lambda df: set(df['subject']),
            preds_dfs.values()
        )
    ))
})

print(f"Subjects: {acc_by_subjects['subject']}")

for run_name, preds_df in preds_dfs.items():
    acc_df = preds_df[['subject', 'corr']].groupby(['subject'], as_index=False).mean()
    acc_df.rename(columns={'corr': run_name}, inplace=True)

    acc_by_subjects = acc_by_subjects.merge(
        acc_df,
        left_on='subject',
        right_on='subject',
        how='right'
    )

Subjects: 0                            econometrics
1                       international_law
2                   professional_medicine
3                           miscellaneous
4                              prehistory
5                            global_facts
6                                virology
7                  high_school_statistics
8                       computer_security
9                         human_sexuality
10                      college_chemistry
11                       medical_genetics
12                        business_ethics
13                 high_school_us_history
14                    high_school_biology
15                              astronomy
16    high_school_government_and_politics
17                       public_relations
18                                anatomy
19                         moral_disputes
20                            human_aging
21                        world_religions
22                    high_school_physics
23             high_scho

In [6]:
acc_by_subjects

,subject,raw 0 shot,raw 5 shot,LoRA 0 shot,LoRA 5 shot,GSOFT_2 0 shot,GSOFT_2 5 shot,GSOFT_3 0 shot,GSOFT_3 5 shot,GSOFT_4 0 shot,GSOFT_4 5 shot,GSOFT_5 0 shot,GSOFT_5 5 shot
0,abstract_algebra,0.350000,0.310000,0.310000,0.340000,0.320000,0.250000,0.240000,0.300000,0.220000,0.310000,0.210000,0.300000
1,anatomy,0.674074,0.666667,0.629630,0.614815,0.644444,0.637037,0.666667,0.651852,0.696296,0.681481,0.688889,0.659259
2,astronomy,0.750000,0.723684,0.717105,0.730263,0.736842,0.703947,0.743421,0.730263,0.763158,0.723684,0.750000,0.736842
3,business_ethics,0.660000,0.670000,0.610000,0.650000,0.690000,0.690000,0.710000,0.680000,0.680000,0.670000,0.710000,0.670000
4,clinical_knowledge,0.716981,0.747170,0.743396,0.769811,0.762264,0.766038,0.762264,0.758491,0.743396,0.750943,0.739623,0.747170
5,college_biology,0.715278,0.756944,0.784722,0.791667,0.763889,0.770833,0.756944,0.763889,0.736111,0.791667,0.750000,0.777778
6,college_chemistry,0.450000,0.460000,0.480000,0.490000,0.480000,0.500000,0.450000,0.510000,0.450000,0.490000,0.450000,0.520000
7,college_computer_science,0.470000,0.570000,0.470000,0.540000,0.420000,0.570000,0.460000,0.580000,0.440000,0.570000,0.460000,0.590000
8,college_mathematics,0.350000,0.350000,0.280000,0.380000,0.300000,0.370000,0.240000,0.360000,0.270000,0.380000,0.290000,0.380000
9,college_medicine,0.618497,0.641618,0.664740,0.647399,0.676301,0.658960,0.647399,0.653179,0.635838,0.647399,0.635838,0.647399


In [7]:
acc_by_categories = pd.DataFrame({
    'category': list(set.union(
        *map(
            lambda df: set(df['category']),
            preds_dfs.values()
        )
    ))
})

for run_name, preds_df in preds_dfs.items():
    acc_df = preds_df[['category', 'corr']].groupby(['category'], as_index=False).mean()
    acc_df.rename(columns={'corr': run_name}, inplace=True)

    acc_by_categories = acc_by_categories.merge(
        acc_df,
        left_on='category',
        right_on='category',
        how='right'
    )

In [8]:
acc_by_categories.loc[:, list(filter(
    lambda s: ('5 shot' not in s),
    acc_by_categories.columns
))]

,category,raw 0 shot,LoRA 0 shot,GSOFT_2 0 shot,GSOFT_3 0 shot,GSOFT_4 0 shot,GSOFT_5 0 shot
0,STEM,0.525182,0.540755,0.530152,0.516567,0.514248,0.519881
1,humanities,0.527099,0.556217,0.559405,0.557705,0.552391,0.552179
2,"other (business, health, misc.)",0.706663,0.709130,0.722702,0.724553,0.722085,0.722394
3,social sciences,0.721807,0.746831,0.758206,0.757556,0.761131,0.761781


In [9]:
acc_by_categories.loc[:, list(filter(
    lambda s: ('0 shot' not in s),
    acc_by_categories.columns
))]

,category,raw 5 shot,LoRA 5 shot,GSOFT_2 5 shot,GSOFT_3 5 shot,GSOFT_4 5 shot,GSOFT_5 5 shot
0,STEM,0.546720,0.561962,0.555335,0.560968,0.561630,0.561299
1,humanities,0.603826,0.591711,0.606376,0.601913,0.596599,0.602550
2,"other (business, health, misc.)",0.724244,0.719926,0.735040,0.735349,0.734115,0.731956
3,social sciences,0.758531,0.766006,0.770556,0.773806,0.775106,0.775106


In [10]:
total_res = pd.DataFrame(columns=['run_name', 'accuracy', 'correctness'])

for run_name, preds_df in preds_dfs.items():
    total_accuracy = preds_df['corr'].mean()
    total_correctness = (preds_df['pred'] != 'I').mean()

    total_res = pd.concat([
        total_res,
        pd.DataFrame({
            'run_name': [run_name],
            'accuracy': [total_accuracy],
            'correctness': [total_correctness],
        })
    ])

C:\Users\Vladimir\AppData\Local\Temp\ipykernel_4756\2066953499.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  total_res = pd.concat([


In [11]:
total_res

,run_name,accuracy,correctness
0,raw 0 shot,0.610810,0.997151
0,raw 5 shot,0.653255,0.999217
0,LoRA 0 shot,0.629967,0.992024
0,LoRA 5 shot,0.653112,1.000000
0,GSOFT_2 0 shot,0.634383,0.988748
0,GSOFT_2 5 shot,0.661088,0.999929
0,GSOFT_3 0 shot,0.631178,0.984902
0,GSOFT_3 5 shot,0.661587,1.000000
0,GSOFT_4 0 shot,0.629113,0.984974
0,GSOFT_4 5 shot,0.659949,1.000000
